# Preparing the Julia Environment

In [1]:
using Pkg; Pkg.activate(".")

  Activating project at `~/code/julia/notebooks`


In [2]:
using JuMP, HiGHS

# Example 1

$$ 
\begin{aligned}
\max\quad & 3 x + 4 y\\
\text{Subject to} \quad & 2 x + 4 y \leq 470\\
 & 6 x + 3 y \leq 360\\
 & x,y \in \{0\} \cup \mathbb{Z}^+ \\
\end{aligned}
$$

In [7]:
begin 
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, x >= 0)
    @variable(model, y >= 0)
    @objective(model, Max, 3*x + 4y)
    @constraint(model, 2*x + 4*y <= 470)
    @constraint(model, 6*x + 3*y <= 360)
    optimize!(model)
    println("x and y: ", value.([x, y]))
    println("objective: ", JuMP.objective_value(model))
end

x and y: [1.6666666666666667, 116.66666666666667]
objective: 471.6666666666667


The relaxed solution is $x = 1.6666$ and $y = 116.6666$, $z = 471.666$. But $x$ and $y$ are supposed to be integer.
Lets apply the Branch and Bound algorithm. Select one of the variables to start. Lets start with $y$:

- if $y$ is an integer, it can not be $166.666$. Lets make branches:
- $y \le 116$
- $y \ge 117$

In [8]:
begin 
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, x >= 0)
    @variable(model, y >= 0)
    @objective(model, Max, 3*x + 4y)
    @constraint(model, 2*x + 4*y <= 470)
    @constraint(model, 6*x + 3*y <= 360)
    @constraint(model, y <= 116)
    optimize!(model)
    println("x and y: ", value.([x, y]))
    println("objective: ", JuMP.objective_value(model))
end

x and y: [2.0, 116.0]
objective: 470.0


In [9]:
begin 
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, x >= 0)
    @variable(model, y >= 0)
    @objective(model, Max, 3*x + 4y)
    @constraint(model, 2*x + 4*y <= 470)
    @constraint(model, 6*x + 3*y <= 360)
    @constraint(model, y >= 117)
    optimize!(model)
    println("x and y: ", value.([x, y]))
    println("objective: ", JuMP.objective_value(model))
end

x and y: [1.0, 117.0]
objective: 471.0


- Both solutions are integers
- The solution corresponding to $y \le 116$ is $x = 2, y = 116, z = 470$
- The solution corresponding to $y \ge 117$ is $x = 1, y = 117, z = 471$
- We pick the second one, since it has integer solutions and has the maximum $z$ value.
- The solution set is $\{x = 1, y = 117\}$

# Example 2

$$ 
\begin{aligned}
\max\quad & 5 x + 4 y\\
\text{Subject to} \quad & x + y \leq 5\\
 & 10 x + 6 y \leq 45\\
 & x,y \in \{0\} \cup \mathbb{Z}^+ \\
\end{aligned} 
$$

In [23]:
begin 
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, x >= 0)
    @variable(model, y >= 0)
    @objective(model, Max, 5*x + 4y)
    @constraint(model, x + y <= 5)
    @constraint(model, 10*x + 6*y <= 45)
    optimize!(model)
    println("x and y: ", value.([x, y]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end

x and y: [3.7499999999999996, 1.250000000000001]
objective: 23.75
Feasible: true


- The relaxed solution is $x = 3.75, y = 1.25, z = 23.75$
- But a set of integer solutions is required.
- Let's branch and bound!
- Let's the branching variable is $x$

## LP0 + (x <= 3)

In [24]:
begin 
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, x >= 0)
    @variable(model, y >= 0)
    @objective(model, Max, 5*x + 4y)
    @constraint(model, x + y <= 5)
    @constraint(model, 10*x + 6*y <= 45)
    @constraint(model, x <= 3)
    optimize!(model)
    println("x and y: ", value.([x, y]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end

x and y: [3.0, 2.0]
objective: 23.0
Feasible: true


## LP0 + (x >= 4)

In [25]:
begin 
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, x >= 0)
    @variable(model, y >= 0)
    @objective(model, Max, 5*x + 4y)
    @constraint(model, x + y <= 5)
    @constraint(model, 10*x + 6*y <= 45)
    @constraint(model, x >= 4)
    optimize!(model)
    println("x and y: ", value.([x, y]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end

x and y: [4.0, 0.8333333333333334]
objective: 23.333333333333332
Feasible: true


## LP0 + (x >= 4) + (y <= 0)

In [26]:
begin 
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, x >= 0)
    @variable(model, y >= 0)
    @objective(model, Max, 5*x + 4y)
    @constraint(model, x + y <= 5)
    @constraint(model, 10*x + 6*y <= 45)
    @constraint(model, x >= 4)
    @constraint(model, y <= 0)
    optimize!(model)
    println("x and y: ", value.([x, y]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end

x and y: [4.5, -0.0]
objective: 22.5
Feasible: true


## LP0 + (x >= 4) + (y >= 1)

In [22]:
begin 
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, x >= 0)
    @variable(model, y >= 0)
    @objective(model, Max, 5*x + 4y)
    @constraint(model, x + y <= 5)
    @constraint(model, 10*x + 6*y <= 45)
    @constraint(model, x >= 4)
    @constraint(model, y >= 1)
    optimize!(model)
    println("x and y: ", value.([x, y]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end

x and y: [6.4e-323, 6.639383970216e-310]
objective: 0.0
Feasible: false


## LP0 + (x >= 4) + (y <= 0) + (x <= 4)

In [27]:
begin 
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, x >= 0)
    @variable(model, y >= 0)
    @objective(model, Max, 5*x + 4y)
    @constraint(model, x + y <= 5)
    @constraint(model, 10*x + 6*y <= 45)
    @constraint(model, x >= 4)
    @constraint(model, y <= 0)
    @constraint(model, x <= 4)
    optimize!(model)
    println("x and y: ", value.([x, y]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end

x and y: [4.0, -0.0]
objective: 20.0
Feasible: true


## LP0 + (x >= 4) + (y <= 0) + (x >= 5)

In [29]:
begin 
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, x >= 0)
    @variable(model, y >= 0)
    @objective(model, Max, 5*x + 4y)
    @constraint(model, x + y <= 5)
    @constraint(model, 10*x + 6*y <= 45)
    @constraint(model, x >= 4)
    @constraint(model, y <= 0)
    @constraint(model, x >= 5)
    optimize!(model)
    println("x and y: ", value.([x, y]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end

x and y: [6.6392715980181e-310, 2.0e-323]
objective: 0.0
Feasible: false


The solution:
- $x = 3$
- $y = 2$
- $z = 23$

# Example 3

|        |  item 1 | item 2 | item 3 | item 4 |
| :---:  | :----:  | :----: | :---:  | :---:  |
| value  |   5     |   7    |    11  |   8    |
| weight |  10     |   12   |    13  |   14   |

Capacity is 30 kgs. Solve the knapsack problem

In [37]:
begin
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, 0 <= x1 <= 1)
    @variable(model, 0 <= x2 <= 1)
    @variable(model, 0 <= x3 <= 1)
    @variable(model, 0 <= x4 <= 1)
    @objective(model, Max, 5*x1 + 7*x2 + 11*x3 + 8*x4)
    @constraint(model, 10*x1 + 12*x2 + 13*x3 + 14*x4 <= 30)
    optimize!(model)
    println("xs: ", value.([x1, x2, x3, x4]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end 

xs: [0.0, 1.0, 1.0, 0.35714285714285715]
objective: 20.857142857142858
Feasible: true


## LP0 + (x4 <= 0)

In [38]:
begin
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, 0 <= x1 <= 1)
    @variable(model, 0 <= x2 <= 1)
    @variable(model, 0 <= x3 <= 1)
    @variable(model, 0 <= x4 <= 1)
    @objective(model, Max, 5*x1 + 7*x2 + 11*x3 + 8*x4)
    @constraint(model, 10*x1 + 12*x2 + 13*x3 + 14*x4 <= 30)
    @constraint(model, x4 <= 0)
    optimize!(model)
    println("xs: ", value.([x1, x2, x3, x4]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end 

xs: [0.5, 1.0, 1.0, -0.0]
objective: 20.5
Feasible: true


## LP0 + (x4 >= 1)

In [40]:
begin
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, 0 <= x1 <= 1)
    @variable(model, 0 <= x2 <= 1)
    @variable(model, 0 <= x3 <= 1)
    @variable(model, 0 <= x4 <= 1)
    @objective(model, Max, 5*x1 + 7*x2 + 11*x3 + 8*x4)
    @constraint(model, 10*x1 + 12*x2 + 13*x3 + 14*x4 <= 30)
    @constraint(model, x4 >= 1)
    optimize!(model)
    println("xs: ", value.([x1, x2, x3, x4]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end 

xs: [0.0, 0.25, 1.0, 1.0]
objective: 20.75
Feasible: true


## LP0 + (x4 <= 0) + (x1 <= 0)

In [41]:
begin
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, 0 <= x1 <= 1)
    @variable(model, 0 <= x2 <= 1)
    @variable(model, 0 <= x3 <= 1)
    @variable(model, 0 <= x4 <= 1)
    @objective(model, Max, 5*x1 + 7*x2 + 11*x3 + 8*x4)
    @constraint(model, 10*x1 + 12*x2 + 13*x3 + 14*x4 <= 30)
    @constraint(model, x4 <= 0)
    @constraint(model, x1 <= 0)
    optimize!(model)
    println("xs: ", value.([x1, x2, x3, x4]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end 

xs: [-0.0, 1.0, 1.0, -0.0]
objective: 18.0
Feasible: true


## LP0 + (x4 <= 0) + (x1 >= 1)

In [46]:
begin
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, 0 <= x1 <= 1)
    @variable(model, 0 <= x2 <= 1)
    @variable(model, 0 <= x3 <= 1)
    @variable(model, 0 <= x4 <= 1)
    @objective(model, Max, 5*x1 + 7*x2 + 11*x3 + 8*x4)
    @constraint(model, 10*x1 + 12*x2 + 13*x3 + 14*x4 <= 30)
    @constraint(model, x4 <= 0)
    @constraint(model, x1 >= 1)
    optimize!(model)
    println("xs: ", value.([x1, x2, x3, x4]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end 

xs: [1.0, 0.5833333333333334, 1.0, 0.0]
objective: 20.083333333333336
Feasible: true


## LP0 + (x4 >= 1) + (x2 <= 0)

In [44]:
begin
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, 0 <= x1 <= 1)
    @variable(model, 0 <= x2 <= 1)
    @variable(model, 0 <= x3 <= 1)
    @variable(model, 0 <= x4 <= 1)
    @objective(model, Max, 5*x1 + 7*x2 + 11*x3 + 8*x4)
    @constraint(model, 10*x1 + 12*x2 + 13*x3 + 14*x4 <= 30)
    @constraint(model, x4 >= 1)
    @constraint(model, x2 <= 0)
    optimize!(model)
    println("xs: ", value.([x1, x2, x3, x4]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end 

xs: [0.3, -0.0, 1.0, 1.0]
objective: 20.5
Feasible: true


## LP0 + (x4 >= 1) + (x2 >= 1)

In [45]:
begin
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, 0 <= x1 <= 1)
    @variable(model, 0 <= x2 <= 1)
    @variable(model, 0 <= x3 <= 1)
    @variable(model, 0 <= x4 <= 1)
    @objective(model, Max, 5*x1 + 7*x2 + 11*x3 + 8*x4)
    @constraint(model, 10*x1 + 12*x2 + 13*x3 + 14*x4 <= 30)
    @constraint(model, x4 >= 1)
    @constraint(model, x2 >= 1)
    optimize!(model)
    println("xs: ", value.([x1, x2, x3, x4]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end 

xs: [0.0, 1.0, 0.3076923076923077, 1.0]
objective: 18.384615384615387
Feasible: true


## LP0 + (x4 <= 0) + (x1 >= 1) + (x2 <= 0)

In [47]:
begin
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, 0 <= x1 <= 1)
    @variable(model, 0 <= x2 <= 1)
    @variable(model, 0 <= x3 <= 1)
    @variable(model, 0 <= x4 <= 1)
    @objective(model, Max, 5*x1 + 7*x2 + 11*x3 + 8*x4)
    @constraint(model, 10*x1 + 12*x2 + 13*x3 + 14*x4 <= 30)
    @constraint(model, x4 <= 0)
    @constraint(model, x1 >= 1)
    @constraint(model, x2 <= 0)
    optimize!(model)
    println("xs: ", value.([x1, x2, x3, x4]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end 

xs: [1.0, -0.0, 1.0, -0.0]
objective: 16.0
Feasible: true


## LP0 + (x4 <= 0) + (x1 >= 1) + (x2 >= 1)

In [48]:
begin
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, 0 <= x1 <= 1)
    @variable(model, 0 <= x2 <= 1)
    @variable(model, 0 <= x3 <= 1)
    @variable(model, 0 <= x4 <= 1)
    @objective(model, Max, 5*x1 + 7*x2 + 11*x3 + 8*x4)
    @constraint(model, 10*x1 + 12*x2 + 13*x3 + 14*x4 <= 30)
    @constraint(model, x4 <= 0)
    @constraint(model, x1 >= 1)
    @constraint(model, x2 >= 1)
    optimize!(model)
    println("xs: ", value.([x1, x2, x3, x4]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end 

xs: [1.0, 1.0, 0.6153846153846154, 0.0]
objective: 18.76923076923077
Feasible: true


## LP0 + (x4 >= 1) + (x2 <= 0) + (x1 <= 0)

In [49]:
begin
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, 0 <= x1 <= 1)
    @variable(model, 0 <= x2 <= 1)
    @variable(model, 0 <= x3 <= 1)
    @variable(model, 0 <= x4 <= 1)
    @objective(model, Max, 5*x1 + 7*x2 + 11*x3 + 8*x4)
    @constraint(model, 10*x1 + 12*x2 + 13*x3 + 14*x4 <= 30)
    @constraint(model, x4 >= 1)
    @constraint(model, x2 <= 0)
    @constraint(model, x1 <= 0)
    optimize!(model)
    println("xs: ", value.([x1, x2, x3, x4]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end 

xs: [-0.0, -0.0, 1.0, 1.0]
objective: 19.0
Feasible: true


## LP0 + (x4 >= 1) + (x2 <= 0) + (x1 >= 1)

In [50]:
begin
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, 0 <= x1 <= 1)
    @variable(model, 0 <= x2 <= 1)
    @variable(model, 0 <= x3 <= 1)
    @variable(model, 0 <= x4 <= 1)
    @objective(model, Max, 5*x1 + 7*x2 + 11*x3 + 8*x4)
    @constraint(model, 10*x1 + 12*x2 + 13*x3 + 14*x4 <= 30)
    @constraint(model, x4 >= 1)
    @constraint(model, x2 <= 0)
    @constraint(model, x1 >= 1)
    optimize!(model)
    println("xs: ", value.([x1, x2, x3, x4]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end 

xs: [1.0, 0.0, 0.46153846153846156, 1.0]
objective: 18.076923076923077
Feasible: true


- The other branches will result with a number less than 19.
- We are stopping iterations here.
- Best found solution is $0, 0, 1, 1$ and $ z = 19$.
- Here is the integer solution with Julia:

In [51]:
begin
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, x1, Bin)
    @variable(model, x2, Bin)
    @variable(model, x3, Bin)
    @variable(model, x4, Bin)
    @objective(model, Max, 5*x1 + 7*x2 + 11*x3 + 8*x4)
    @constraint(model, 10*x1 + 12*x2 + 13*x3 + 14*x4 <= 30)
    optimize!(model)
    println("xs: ", value.([x1, x2, x3, x4]))
    println("objective: ", JuMP.objective_value(model))
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
end 

xs: [0.0, 0.0, 1.0, 1.0]
objective: 19.0
Feasible: true
